In [ ]:
import pandas as pd
import re
import time
# from nltk.corpus import words
from nltk.tokenize import word_tokenize
import nltk
#nltk.download('punkt')
from textblob import Word
pd.options.display.width = 0

import spacy
nlp = spacy.load('en_core_web_sm')
all_stopwords = nlp.Defaults.stop_words

pd.set_option('display.max_rows', 6000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 1000)
#pd.describe_option('display')
# import enchant

In [ ]:
past_time = time.time()

In [ ]:
path='/nfs/home/vyasa/projects/proj_off/data_off/clarify/spanish_comorbidity/new/'

In [ ]:
df = pd.read_csv(path+'comorbidities_modified_witht_abbr.csv', sep=',', error_bad_lines=True, encoding='utf-8')
df.shape

In [ ]:
df.head(5)

In [ ]:
df['IsValueReplaced'] = df['value']!=df['value_witht_abbr']
cols_l = df.columns.tolist()
df_new = df[cols_l[0:2]+cols_l[-1:]+cols_l[2:-1]]
df = df_new

In [ ]:
df['replacement'] = df['replacement'].str.replace(';', '')
df['replacement'] = df['replacement'].str.strip()

In [ ]:
df.columns

In [ ]:
correction_dict = {'Benign BIRADS2 breast nodule': 'Bening breast nodule (BI-RADS)',
 'HT+ DA cavit': 'Hormone therapy',
 'Bilateral hydrosadenitis': 'Hidradenitis suppurativa',
 'SCASEST': 'Acute coronary syndrome',
 'IAM': 'Acute Myocardial Infarction',
 'Alcholic': 'Alcoholic',
 'Hypoacucia': 'Hearing loss',
 'sd IAVD': 'Right ventricle Acute myocardial infarction',
 'Saphenous ischemia': 'Saphenous vein Ischemia'}

In [ ]:
df.replace({'replacement':correction_dict}, inplace=True)

In [ ]:
# Find Spanish Abbreviations

In [ ]:
df_sp_google = pd.read_csv(path+'comorbidities_spanish_english_google.csv', sep=',', error_bad_lines=True, encoding='utf-8')
print (df_sp_google.shape)

df_sp_google_abbr = pd.read_csv(path+'comorbidities_value_abbr_google.csv', sep=',', error_bad_lines=True, encoding='utf-8')
print (df_sp_google.shape)



In [ ]:
df_sp_deep = pd.read_csv(path+'comorbidities_spanish_english_deepl.csv', sep=',', error_bad_lines=True, encoding='utf-8')
print (df_sp_deep.shape)

df_sp_deep_abbr = pd.read_csv(path+'comorbidities_value_abbr_deepl.csv', sep=',', error_bad_lines=True, encoding='utf-8')
print (df_sp_deep.shape)

In [ ]:
df['google'] = df_sp_google['value']
df['deepl'] = df_sp_deep['value']
df['google_abbr'] = df_sp_google_abbr['value_witht_abbr_google']
df['deepl_abbr'] = df_sp_deep_abbr['value_witht_abbr_deepl']

In [ ]:
df['google'] = df['google'].str.strip()
df['deepl'] = df['deepl'].str.strip()
df['google_abbr'] = df['google_abbr'].str.strip()
df['deepl_abbr'] = df['deepl_abbr'].str.strip()

In [ ]:
def capitalize_(text):
    if type(text)==str:
        if len(text)>0:
            # return text[0:1].capitalize()+text[1:].lower()
            return ' '.join([i[0:1].upper()+i[1:] if len(i)>1 else i for i in text.split()])
    else:
        return text

df['replacement'] = df['replacement'].apply(capitalize_)
df['google'] = df['google'].apply(capitalize_)
df['deepl'] = df['deepl'].apply(capitalize_)
df['google_abbr'] = df['google_abbr'].apply(capitalize_)
df['deepl_abbr'] = df['deepl_abbr'].apply(capitalize_)

In [ ]:
df.head(100)

In [ ]:
# Remove stop words and years
def remove_stopwords_numbers(text):
    return ' '.join([word for word in word_tokenize(str(text)) \
        if word not in all_stopwords and not word.isdigit()])

df['google_withst_num'] = df['google'].apply(remove_stopwords_numbers)
df['deepl_withst_num'] = df['deepl'].apply(remove_stopwords_numbers)
df['google_abbr_withst_num'] = df['google_abbr'].apply(remove_stopwords_numbers)
df['deepl_abbr_withst_num'] = df['deepl_abbr'].apply(remove_stopwords_numbers)

In [ ]:
# Calculate Abbreviations
def find_in_english_dict(text):
    for w in word_tokenize(text):
        if Word(w.strip().lower()).spellcheck()[0][1]!=1.0 and len(re.findall(r"[A-Z]{2,}", w))>0 \
            and Word(w.strip().lower()).spellcheck()[0][0]!=w.strip().lower():
            print ("True",w)
          # print(w)
            return True
    return False

df['haveAbbr'] = (df['value_witht_abbr'].apply(lambda x : len(re.findall(r"[A-Z]{2,}", x))>0)) & \
    (df['google'].apply(find_in_english_dict))

In [ ]:
# Calculate no of stop words
def stop_words_length(text):
    # print (word_tokenize(str(text)))
    return len([word for word in word_tokenize(str(text)) if word in all_stopwords])

df['stop_words_google_len'] = df['google'].apply(stop_words_length)
df['stop_words_deepl_len'] = df['deepl'].apply(stop_words_length)
df['stop_words_google_abbr_len'] = df['google_abbr'].apply(stop_words_length)
df['stop_words_deepl_abbr_len'] = df['deepl_abbr'].apply(stop_words_length)

In [ ]:
df.to_csv(path+'spanish_google_deep_last.csv', index=False, encoding='utf-8')

In [ ]:
df[df['haveAbbr']==True].to_csv(path+'spanish_google_deep_last_index.csv', index=True, encoding='utf-8')

In [ ]:
df.head(5)

In [ ]:
df.tail(5)

In [ ]:
# Train_data
df_MEV = df.dropna(subset=['replacement'])
df_MEV.to_csv(path+'train_MEV.csv', index=False, encoding='utf-8')

In [ ]:
# df_MEV.reset_index().iloc[[60,67,132,160,163,444,446,462,467,478,507,511,573,675,677,721,834]]['replacement'].tolist()

In [ ]:
df_MEV.head(50)

In [ ]:
print ("Total_Time ({}(in Hrs) : {}(in Mins)".format((time.time()-past_time)/3600.0, (time.time()-past_time)/60.0))